<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
%pip install plotly
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [ ]:
import folium
import piplite
import pandas as pd
import plotly.express as px

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map
# Load sample earthquake data
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')
df = df.dropna(subset=['latitude', 'longitude', 'mag'])
# Create a scatter geo map
fig = px.scatter_geo(df,
                     lat='latitude',
                     lon='longitude',
                     color='mag',
                     hover_name='place',
                     title='Earthquakes Around the World',
                     color_continuous_scale='reds')
fig.show()


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [ ]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterate over each launch record to plot outcome markers
for _, row in spacex_df.iterrows():
    outcome = 'Success' if row['class'] == 1 else 'Failure'
    color = 'green' if row['class'] == 1 else 'red'
    icon = 'ok-sign' if row['class'] == 1 else 'remove-sign'

    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=folium.Popup(f"{row['Launch Site']}<br>{outcome}", parse_html=True),
        icon=folium.Icon(color=color, icon=icon, prefix='glyphicon')
    ).add_to(site_map)

# Group by site and compute success rate
success_df = spacex_df.groupby('Launch Site')['class'].agg(['mean', 'count']).reset_index()
success_df.rename(columns={'mean': 'Success Rate', 'count': 'Total Launches'}, inplace=True)

# Merge with geo coordinates
launch_stats_df = pd.merge(success_df, launch_sites_df, on='Launch Site')
# Create the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circle for each site with color based on success rate
for _, row in launch_stats_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=row['Total Launches'] * 200,  # Size reflects launch volume
        color='green' if row['Success Rate'] > 0.8 else 'orange' if row['Success Rate'] > 0.5 else 'red',
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(f"{row['Launch Site']}<br>Success Rate: {row['Success Rate']:.2f}<br>Total Launches: {int(row['Total Launches'])}", max_width=300)
    ).add_to(site_map)


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
spacex_df.tail(10)

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
#marker_cluster = MarkerCluster()
from folium.plugins import MarkerCluster

# Create base map centered at NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create the marker cluster object
marker_cluster = MarkerCluster().add_to(site_map)
# Loop through each launch record
for _, row in spacex_df.iterrows():
    outcome = 'Success' if row['class'] == 1 else 'Failure'
    color = 'green' if row['class'] == 1 else 'red'
    icon = 'ok-sign' if row['class'] == 1 else 'remove-sign'

    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']}<br>{outcome}",
        icon=folium.Icon(color=color, icon=icon, prefix='glyphicon')
    ).add_to(marker_cluster)




*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Assign marker colors based on launch outcome
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
folium.Icon(color=row['marker_color'], icon='rocket', prefix='fa')


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
#site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
#for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
 #   marker_cluster.add_child(marker)

#site_map
# Create the base map and MarkerCluster
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for each launch
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']}<br>{'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'], icon='rocket', prefix='fa')
    ).add_to(marker_cluster)


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Haversine function to calculate distance in kilometers
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c
# NASA JSC coordinate
nasa_lat, nasa_lon = nasa_coordinate

# Calculate distances to each launch in spacex_df
spacex_df['distance_to_NASA'] = spacex_df.apply(
    lambda row: haversine(nasa_lat, nasa_lon, row['Lat'], row['Long']),
    axis=1
)
# Sort by distance
closest_launch = spacex_df.sort_values('distance_to_NASA').head(1)
farthest_launch = spacex_df.sort_values('distance_to_NASA').tail(1)


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
# Create the base map centered on NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# Add MousePosition to display coordinates on hover
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
mouse_position.add_to(site_map)


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius (km)
    from math import radians, sin, cos, sqrt, asin
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return R * 2 * asin(sqrt(a))
# NASA JSC to a nearby rail point
distance_km = haversine(29.559684888503615, -95.0830971930759, 29.5610, -95.0870)
print(f"Distance: {distance_km:.2f} km")


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
def haversine(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, sqrt, asin
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    return R * 2 * asin(sqrt(a))

# Example usage
launch_site = (38.4761, 7.0283)  # Replace with your actual coordinates
coastline_point = (38.5100, 6.9700)
distance = haversine(*launch_site, *coastline_point)
print(f"Distance to coastline: {distance:.2f} km")


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
import folium

# Replace these with the actual coordinates you got from MousePosition
coastline_lat = 38.5100
coastline_lon = 6.9700

# Initialize your map centered on the launch site (just an example)
m = folium.Map(location=[38.4761, 7.0283], zoom_start=12)

# Add coastline marker
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup="Closest Coastline Point",
    icon=folium.Icon(color="blue", icon="cloud")
).add_to(m)

# Display the map
m


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)
# Example coordinates – replace these with your actual data
launch_site = [38.4761, 7.0283]
coastline_point = [38.5100, 6.9700]

# Initialize map centered roughly between the two points
center_lat = (launch_site[0] + coastline_point[0]) / 2
center_lon = (launch_site[1] + coastline_point[1]) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Add marker for Launch Site
folium.Marker(
    location=launch_site,
    popup="Launch Site",
    icon=folium.Icon(color="red", icon="rocket")
).add_to(m)

# Add marker for Coastline Point
folium.Marker(
    location=coastline_point,
    popup="Closest Coastline Point",
    icon=folium.Icon(color="blue", icon="cloud")
).add_to(m)

# Add PolyLine between the two points
folium.PolyLine(
    locations=[launch_site, coastline_point],
    color="green",
    weight=3,
    tooltip="Distance Line"
).add_to(m)

# Display the map
m


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [ ]:
import folium
from folium.plugins import MousePosition

# Launch site coordinates (example: Hawassa)
launch_site = [7.0621, 38.4764]

# Coordinates from MousePosition (replace with actual values you collect)
closest_city = [6.8667, 38.4000]     # Example: Shashamene
closest_railway = [8.5500, 39.2667]  # Example: Adama station
closest_highway = [7.0700, 38.4800]  # Example: Addis–Hawassa Highway

# Create map centered on launch site
m = folium.Map(location=launch_site, zoom_start=10)

# Add MousePosition plugin
MousePosition().add_to(m)

# Add markers
folium.Marker(launch_site, popup="Launch Site", icon=folium.Icon(color="red", icon="rocket")).add_to(m)
folium.Marker(closest_city, popup="Closest City", icon=folium.Icon(color="purple", icon="home")).add_to(m)
folium.Marker(closest_railway, popup="Closest Railway", icon=folium.Icon(color="green", icon="train")).add_to(m)
folium.Marker(closest_highway, popup="Closest Highway", icon=folium.Icon(color="blue", icon="road")).add_to(m)

# Draw lines
folium.PolyLine([launch_site, closest_city], color="orange", weight=2.5, tooltip="To City").add_to(m)
folium.PolyLine([launch_site, closest_railway], color="darkgreen", weight=2.5, tooltip="To Railway").add_to(m)
folium.PolyLine([launch_site, closest_highway], color="navy", weight=2.5, tooltip="To Highway").add_to(m)

# Display map
m


In [ ]:
import folium

# Coordinates from MousePosition (replace with your actual values)
launch_site = [7.0621, 38.4764]         # Hawassa
marker_point = [6.8667, 38.4000]        # Example: Shashamene (closest city)

# Create map centered between the two
center_lat = (launch_site[0] + marker_point[0]) / 2
center_lon = (launch_site[1] + marker_point[1]) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add launch site marker
folium.Marker(
    location=launch_site,
    popup="Launch Site",
    icon=folium.Icon(color="red", icon="rocket")
).add_to(m)

# Add infrastructure marker
folium.Marker(
    location=marker_point,
    popup="Closest City",
    icon=folium.Icon(color="purple", icon="home")
).add_to(m)

# Draw line between them
folium.PolyLine(
    locations=[launch_site, marker_point],
    color="orange",
    weight=3,
    tooltip="Launch Site → City"
).add_to(m)

# Display map
m


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
